In [ ]:
import numpy as np
import random
import numpy.linalg as npl
from matplotlib import pyplot as plt

In [ ]:
num_clients=1000
base_s=5
data_A=[]
data_b=[]
client_s=[]
iid=True
n=10
d=10
for i in range(num_clients):
    if iid:
        client_s.append(base_s)
    else:
        client_s.append(abs(np.random.uniform(0,2*base_s)))
    s=client_s[-1]
    b = np.random.normal(0,base_s**2,n)
    #print(np.mean(b))
    b=b-np.mean(b)

    #A = np.diag(np.random.uniform(0,1.,n))

    A = np.diag(np.random.normal(0,1**2.,n))

    # A = np.eye(n)*np.random.uniform(0,1.)

    # a=np.random.normal(0,s**2.,n)
    # a[a<1]=1
    # A=np.diag(a)


    #A = np.random.uniform(0,1,(n,n))
    data_A.append(A)
    data_b.append(b)

In [ ]:
select_client=100
num_epochs=10000
lmbda=0.1
lr=0.1
hlr=lr
tau=3
T=3

In [ ]:
result_joint=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
for _ in range(num_epochs):
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        # b = np.random.normal(0,base_s**2,n)
        # A = np.diag(np.random.uniform(0,1.,n))
        # inner optimization
        yi=y.copy()
        xi=x.copy()
        for _ in range(T):
            yi+=lr*(1./n)*( - yi + 0.5*b - 0.5*A@xi)
        for _ in range(tau):
            xi-=hlr*(1./n)*( - 0.5*yi.T@A + lmbda*xi)
        x_clients[i]=xi
        y_clients[i]=yi
    
    x=np.mean(x_clients,axis=0)
    y=np.mean(y_clients,axis=0)
    
    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x
        
    loss/=select_client
    result_joint.append([npl.norm(x)**2,npl.norm(y)**2,loss])
result_joint=np.array(result_joint)
plt.plot(result_joint[:,0])
plt.plot(result_joint[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")

In [ ]:
result_fednest=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
for _ in range(num_epochs):
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    gyi_svrg = np.zeros(n, dtype=np.float32)*10
    gxi_svrg = np.zeros(d, dtype=np.float32)*10

    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        gyi_svrg+=lr*(1./n)*( - y + 0.5*b - 0.5*A@x)
    gyi_svrg/=select_client

    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=y.copy()
        xi=x.copy()
        for _ in range(T):
            yi+=(lr*0.1*( - yi + 0.5*b - 0.5*A@xi) - lr*0.1*( - y + 0.5*b - 0.5*A@x) + gyi_svrg)
        y_clients[i]=yi
    y=np.mean(y_clients,axis=0)

    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        gxi_svrg+=hlr*0.1*( - 0.5*y.T@A + lmbda*x)
    gxi_svrg/=select_client
        
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=y.copy()
        xi=x.copy()
        for _ in range(tau):
            xi-=(hlr*0.1*( - 0.5*yi.T@A + lmbda*xi)-hlr*0.1*( - 0.5*y.T@A + lmbda*x)+gxi_svrg)
        x_clients[i]=xi
    #print(x_clients)
    x=np.mean(x_clients,axis=0)
    
    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x
    loss/=select_client
    
    result_fednest.append([npl.norm(x)**2,npl.norm(y)**2,loss])
result_fednest=np.array(result_fednest)
plt.plot(result_fednest[:,0])
plt.plot(result_fednest[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")

In [ ]:
result_lfednest=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
for _ in range(num_epochs):
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)

    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=y.copy()
        xi=x.copy()
        for _ in range(T):
            yi+=lr*0.1*( - yi + 0.5*b - 0.5*A@xi)
        y_clients[i]=yi
    y=np.mean(y_clients,axis=0)
        
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=y.copy()
        xi=x.copy()
        for _ in range(tau):
            xi-=hlr*0.1*( - 0.5*yi.T@A + lmbda*xi)
        x_clients[i]=xi
    x=np.mean(x_clients,axis=0)

    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x

    loss/=select_client
    result_lfednest.append([npl.norm(x)**2,npl.norm(y)**2,loss])
result_lfednest=np.array(result_lfednest)
plt.plot(result_lfednest[:,0])
plt.plot(result_lfednest[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")

In [ ]:
plt.plot(result_joint[:,0])
plt.plot(result_lfednest[:,0])
plt.plot(result_fednest[:,0])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
plt.yscale('log')


In [ ]:
plt.plot(result_joint[:,1])
plt.plot(result_lfednest[:,1])
plt.plot(result_fednest[:,1])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
plt.yscale('log')

In [ ]:
plt.plot(result_joint[:,2])
plt.plot(result_lfednest[:,2])
plt.plot(result_fednest[:,2])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
